In [ ]:
import wikipedia
from markdownify import markdownify
from tensorzero import AsyncTensorZeroGateway, ToolCall, ToolResult

In [ ]:
MAX_INFERENCES = 20

In [ ]:
t0 = await AsyncTensorZeroGateway.build_embedded(
    config_file="config/tensorzero.toml",
    # clickhouse_url="...",
)

In [ ]:
def search_wikipedia(tool_call: ToolCall) -> ToolResult:
    """
    Searches Wikipedia for a given query and returns a list of search results.

    Args:
        tool_call (ToolCall): A tool call object containing the search query in its arguments.
            Expected arguments: {"query": str}

    Returns:
        ToolResult: A tool result containing the newline-separated list of Wikipedia search results.
            The result field contains the search results as a string.
    """
    search_wikipedia_result = "\n".join(wikipedia.search(tool_call.arguments["query"]))

    return ToolResult(
        name="search_wikipedia",
        id=tool_call.id,
        result=search_wikipedia_result,
    )


def load_wikipedia_page(tool_call: ToolCall) -> ToolResult:
    """
    Loads and formats the content of a Wikipedia page.

    Args:
        tool_call (ToolCall): A tool call object containing the page title in its arguments.
            Expected arguments: {"title": str}

    Returns:
        ToolResult: A tool result containing the formatted Wikipedia page content.
            The result field contains the page URL and content in Markdown format.
            If the page is not found or there's a disambiguation error, returns an error message.
    """
    try:
        page = wikipedia.page(tool_call.arguments["title"])
        # Preprocess result by converting the HTML content to Markdown to reduce token usage
        page_markdown = markdownify(page.html())
        load_wikipedia_page_result = (
            f"# URL\n\n{page.url}\n\n# CONTENT\n\n{page_markdown}"
        )
    except wikipedia.exceptions.PageError:
        load_wikipedia_page_result = (
            f"ERROR: page '{tool_call.arguments['title']}' not found."
        )
    except wikipedia.exceptions.DisambiguationError as e:
        load_wikipedia_page_result = (
            f"ERROR: disambiguation error for '{tool_call.arguments['title']}': {e}"
        )

    return ToolResult(
        name="load_wikipedia_page",
        id=tool_call.id,
        result=load_wikipedia_page_result,
    )

In [ ]:
async def ask_question(question: str, verbose: bool = False):
    """
    Asks a question to the multi-hop retrieval agent and returns the answer.

    Args:
        question (str): The question to ask the agent.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        str: The answer to the question.
    """
    messages = [{"role": "user", "content": question}]

    for _ in range(MAX_INFERENCES):
        print()
        response = await t0.inference(
            function_name="multi_hop_rag_agent", input={"messages": messages}
        )

        messages.append({"role": "assistant", "content": response.content})

        output_content_blocks = []

        for content_block in response.content:
            if isinstance(content_block, ToolCall):
                if verbose:
                    print(
                        f"[Tool Call] {content_block.name}: {content_block.arguments}"
                    )

                if content_block.name is None or content_block.arguments is None:
                    output_content_blocks.append(
                        ToolResult(
                            name=content_block.raw_name,
                            id=content_block.id,
                            result="ERROR: invalid tool call",
                        )
                    )
                elif content_block.name == "search_wikipedia":
                    output_content_blocks.append(search_wikipedia(content_block))
                elif content_block.name == "load_wikipedia_page":
                    output_content_blocks.append(load_wikipedia_page(content_block))
                elif content_block.name == "think":
                    # The `think` tool is just used to plan the next steps, and there's no actual tool to call.
                    # Some providers like OpenAI require a tool result, so we'll provide an empty string.
                    output_content_blocks.append(
                        ToolResult(
                            name="think",
                            id=content_block.id,
                            result="",
                        )
                    )
                elif content_block.name == "answer_question":
                    return content_block.arguments["answer"]
            else:
                print(f"[Other Content Block] {content_block}")

        messages.append({"role": "user", "content": output_content_blocks})
    else:
        raise Exception(f"Failed to answer question after {MAX_INFERENCES} inferences.")

In [ ]:
answer = await ask_question(
    "What is a common dish in the hometown of the scientist that won the Nobel Prize for the discovery of the positron?",
    verbose=True,
)

# answer = await ask_question(
#     "Who is the mayor of the city where the composer of the national anthem of the country with the lowest infant mortality rate was born?", verbose=True
# )

# answer = await ask_question(
#     "What company developed the popular Chinese video game voiced by the same voice actor that voiced a wizard in the anime Konosuba?",
#     verbose=True,
# )

print(
    "┌──────────────────────────────────────────────────────────────────────────────┐\n"
    "│                                 FINAL ANSWER                                 │\n"
    "└──────────────────────────────────────────────────────────────────────────────┘\n"
)
print(answer)